<a href="https://colab.research.google.com/github/oenereaty/MP2024/blob/main/MP2024_wk11_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# Imports

In [2]:
import os
os.environ["WANDB_MODE"] = "offline"

In [3]:
import pandas as pd
import numpy as np
import datasets
import transformers
import torch
import torchvision
import torch.utils
import evaluate

## 1. `data_collator` 의 이해

`data_collator`

데이터콜렉터 (더 전처리 된 데이터)
목적: 인공지능이 학습하기 쉽게 행렬의 형태로 만들기.

tokenizer의 전처리 이후 실행하므로 숫자화(O), `datacollator`로 인해 행렬화한다.

`torch.tensor`, `미니배치`, `패딩/동적패딩` 모두 O. (당연함. 인공지능의 학습을 위해 구조를 떠먹어줘야 하기 때문.)

`datacollator`를 위해 `dictionary`의 형태로 바꿔줘야 함.(label, text, input_ids, attention_mask)

## A. 외워(🥺🥺)

```Python
trainer_input = ~~~
model = ~~~~
#---#
batch_maker = transformers.Trainer(
    model = model,
    data_collator = lambda x: x
) # 이 과정에서 model이 cuda로 감
_batched_data = batch_maker.get_test_dataloader(trainer_input) # 이 과정에서 trainer_input이 cuda로 감
batched_data = list(_batched_data) # `_batched_data`가 list화되면 batched_data
single_batch = batched_data[-1]
model.to("cpu") # 경우에 따라 생략해야할수도있음
model(**data_collator(single_batch))
```

```python
trainer_input = 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22

batch_size=3

batched_data = [
  [1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12], [13, 14, 15], [16, 17, 18], [19, 20, 21], [22]
]

single_data = batcheda_data[0] == [1,2,3]
```

In [4]:
trainer = transformers.Trainer(
    model = model,
    data_collator = data_collator
)
trainer.predict(trainer_input)

NameError: name 'model' is not defined

## B.`IMDB` 복습

In [5]:
# 데이터 준비
imdb = datasets.load_dataset("guebin/imdb-tiny")
tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)
tokenized_imdb = imdb.map(preprocess_function,batched=True)
trainer_input = tokenized_imdb['train']
trainer_input

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/365 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 10
})

In [6]:
# 모델 생성
model = transformers.AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# 데이터 콜렉터
data_collator = transformers.DataCollatorWithPadding(tokenizer=tokenizer)
data_collator

DataCollatorWithPadding(tokenizer=DistilBertTokenizerFast(name_or_path='distilbert/distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, pad_to_multiple_of=None, return_tenso

In [8]:
trainer = transformers.Trainer(
    model = model,
    data_collator = data_collator
)
trainer.predict(trainer_input)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


PredictionOutput(predictions=array([[ 0.01617771, -0.00176518],
       [ 0.03863558,  0.02374268],
       [ 0.07496815,  0.01065765],
       [ 0.05714396,  0.0015379 ],
       [ 0.04593895,  0.03226093],
       [ 0.05797766, -0.00115458],
       [ 0.01432376, -0.00093423],
       [ 0.03092685,  0.03170963],
       [ 0.06981145,  0.04833814],
       [ 0.04450498, -0.00905576]], dtype=float32), label_ids=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), metrics={'test_loss': 0.677581250667572, 'test_model_preparation_time': 0.0069, 'test_runtime': 3.0095, 'test_samples_per_second': 3.323, 'test_steps_per_second': 0.665})

In [9]:
trainer.predict(trainer_input)

PredictionOutput(predictions=array([[ 0.01617771, -0.00176518],
       [ 0.03863558,  0.02374268],
       [ 0.07496815,  0.01065765],
       [ 0.05714396,  0.0015379 ],
       [ 0.04593895,  0.03226093],
       [ 0.05797766, -0.00115458],
       [ 0.01432376, -0.00093423],
       [ 0.03092685,  0.03170963],
       [ 0.06981145,  0.04833814],
       [ 0.04450498, -0.00905576]], dtype=float32), label_ids=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), metrics={'test_loss': 0.677581250667572, 'test_model_preparation_time': 0.0069, 'test_runtime': 0.1981, 'test_samples_per_second': 50.487, 'test_steps_per_second': 10.097})

In [10]:
batch_maker = transformers.Trainer(          # 얘도 어떻게 보면 `Trainer`라고 볼 수 있는데, 이상한 형식의 `Trainer`임
    model = model,
    data_collator = lambda x: x
) # 이 과정에서 model이 cuda로 감
_batched_data = batch_maker.get_test_dataloader(trainer_input) # 이 과정에서 trainer_input이 cuda로 감
batched_data = list(_batched_data)
single_batch = batched_data[-1]
model.to("cpu") # 경우에 따라 생략해야할수도있음
model(**data_collator(single_batch))

SequenceClassifierOutput(loss=tensor(0.6746, grad_fn=<NllLossBackward0>), logits=tensor([[ 0.0698,  0.0483],
        [ 0.0445, -0.0091]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

돌아간다 ⟶ 잘 설계된 `data_collator`라는 의미

In [11]:
trainer = transformers.Trainer(
    model = model,
    data_collator = data_collator
)
out = trainer.predict(trainer_input)
out

PredictionOutput(predictions=array([[ 0.01617771, -0.00176518],
       [ 0.03863558,  0.02374268],
       [ 0.07496815,  0.01065765],
       [ 0.05714396,  0.0015379 ],
       [ 0.04593895,  0.03226093],
       [ 0.05797766, -0.00115458],
       [ 0.01432376, -0.00093423],
       [ 0.03092685,  0.03170963],
       [ 0.06981145,  0.04833814],
       [ 0.04450498, -0.00905576]], dtype=float32), label_ids=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), metrics={'test_loss': 0.677581250667572, 'test_model_preparation_time': 0.0027, 'test_runtime': 0.2079, 'test_samples_per_second': 48.108, 'test_steps_per_second': 9.622})

In [12]:
len(batched_data[-1])

2

In [13]:
imdb
# batch_size=8, `num_rows`=10 이므로
# batch_data = [[1,2,3,4,5,6,7,8], [9,10]]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10
    })
})

In [14]:
# batched_data[0]

`batched_data`로는 `model`에 넣을 수 없다

당연히 `tensor화`도 안 되어 있고... 이상하게 생겼어..

In [15]:
data_collator(batched_data[-1]) # 모델의 입력으로 매우 바람직해 보이는 형식

{'input_ids': tensor([[  101,  2040,  2024,  ..., 22132,  7847,   102],
        [  101,  2023,  2003,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([0, 0])}

- 관찰2: `data_collator(batched_data[-1])` 역시 하나의배치(single_batch)를 의미함. 또한 이것은 모델의 입력으로도 적절한 형식.

In [16]:
model.to("cpu")  # model을 cpu에 넣는 게 더 빠른가 보다🧐
model(**data_collator(batched_data[1]))

SequenceClassifierOutput(loss=tensor(0.6746, grad_fn=<NllLossBackward0>), logits=tensor([[ 0.0698,  0.0483],
        [ 0.0445, -0.0091]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [17]:
model(**data_collator(batched_data))

AttributeError: 'list' object has no attribute 'keys'

`batched_data`는 `list`이므로 `batched_data[0]` 또는 `batched_data[1]` 의 형태로 작성해야 한다

## C. FOOD101 – 복습


In [18]:
food = datasets.load_dataset("guebin/food101-tiny")
image_processor = transformers.AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
normalize = torchvision.transforms.Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop(size),
    torchvision.transforms.ToTensor(),
    normalize
])
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples
trainer_input = food['train'].with_transform(transforms)
trainer_input

README.md:   0%|          | 0.00/3.12k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/499k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10 [00:00<?, ? examples/s]

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


Dataset({
    features: ['image', 'label'],
    num_rows: 10
})

In [19]:
labels = food["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label
model = transformers.AutoModelForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
data_collator = transformers.DefaultDataCollator()
data_collator

DefaultDataCollator(return_tensors='pt')

In [21]:
food["train"]
# jpg

Dataset({
    features: ['image', 'label'],
    num_rows: 10
})

In [22]:
trainer_input
# pixel_values

Dataset({
    features: ['image', 'label'],
    num_rows: 10
})

Q. 밑의 코드가 정상적으로 돌아가는지 확인하여라 → 실패!

In [23]:
trainer.predict(trainer_input)

KeyError: 'image'

In [24]:
batch_maker = transformers.Trainer(
    model = model,
    data_collator = lambda x: x
)
_batched_data = batch_maker.get_test_dataloader(trainer_input)
batched_data = list(_batched_data)
single_batch = batched_data[-1]
model(**data_collator(single_batch))

KeyError: 'image'

다른 방법

In [25]:
trainer_input

Dataset({
    features: ['image', 'label'],
    num_rows: 10
})

`with_transform` 이 지연 실행되는 바람에 `image` 가 삭제 됨

→ 미리 `with_transform`을 실행시켜 버리자

In [26]:
# [trainer_input[0], trainer_input[1], trainer_input[2], ... , trainer_input[9]]
trainer_input2 = [l for l in trainer_input]
#trainer_input2

다 미리 실행 시켜버리자ㅠ 리스트 걍 만들어버려

In [27]:
batch_maker = transformers.Trainer(
    model = model,
    data_collator = lambda x: x
)
_batched_data = batch_maker.get_test_dataloader(trainer_input2)
batched_data = list(_batched_data)
single_batch = batched_data[-1]
model(**data_collator(single_batch))

ImageClassifierOutput(loss=tensor(4.7298, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[-0.0676,  0.1459, -0.1175, -0.1056, -0.0385,  0.0683, -0.1158, -0.1558,
          0.0525, -0.0200,  0.0376, -0.0063, -0.0155, -0.1117, -0.2427, -0.1596,
         -0.1851, -0.0097,  0.0415,  0.0059, -0.0049, -0.2989,  0.2192,  0.0929,
         -0.0221,  0.0580,  0.0327, -0.2149, -0.0519, -0.0122,  0.1131, -0.0620,
         -0.0930,  0.2360,  0.0717, -0.1277,  0.2066,  0.0503,  0.0279,  0.2441,
          0.0090,  0.2275,  0.0816,  0.0550, -0.0890, -0.0293,  0.0727, -0.2700,
          0.1172,  0.0787, -0.0140,  0.1297, -0.0702, -0.1394, -0.0369, -0.0392,
          0.1238, -0.0254,  0.1423, -0.1449, -0.1164, -0.1531, -0.0225,  0.0008,
          0.1119,  0.1147, -0.0927, -0.1213,  0.0792,  0.1642, -0.0733,  0.0432,
          0.0861,  0.1459,  0.1758, -0.0242,  0.1205, -0.0705, -0.0217, -0.0833,
         -0.0036,  0.0387,  0.0447,  0.1393,  0.0777,  0.0186,  0.0297,  0.0184,
          0.09

`model`이 돌아가는 걸 보니 `data_collator`도 잘 짜여졌고,,,

이제 학습시키자!(=`train`)

In [28]:
trainer = transformers.Trainer(
    model = model,
    data_collator= data_collator
)
trainer.predict(trainer_input2)

PredictionOutput(predictions=array([[ 0.08245477,  0.04544435, -0.02506349, ..., -0.09021501,
        -0.06471942,  0.01008125],
       [-0.08817462,  0.07300131,  0.0024789 , ..., -0.02228859,
        -0.14305958,  0.16596188],
       [ 0.07862606, -0.01995676,  0.05375681, ..., -0.13145116,
        -0.06238776,  0.01560452],
       ...,
       [-0.11510422,  0.02849287,  0.01068855, ..., -0.02876251,
        -0.05844339,  0.18115753],
       [-0.06757909,  0.14586404, -0.1175009 , ..., -0.01018211,
        -0.18151312,  0.11967795],
       [-0.19622399,  0.15283908, -0.07533113, ..., -0.01906088,
        -0.13519949,  0.03490356]], dtype=float32), label_ids=array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6]), metrics={'test_loss': 4.646465301513672, 'test_model_preparation_time': 0.0032, 'test_runtime': 0.1783, 'test_samples_per_second': 56.083, 'test_steps_per_second': 11.217})

또 다른 풀이

Q. 꼭 안 쓰는 컬럼들을 없애야 돼??????ㅠㅠㅠㅠㅠㅠㅠㅠㅠ


  ㄴ 아니

In [29]:
trainer_input

Dataset({
    features: ['image', 'label'],
    num_rows: 10
})

In [30]:
batch_maker = transformers.Trainer(
    model = model,
    data_collator = lambda x: x,
    args = transformers.TrainingArguments(
        output_dir="똥개",                    # `output_dir` 은 꼭 써야하지만, 아무거나 써도 ㄱㅊ
        remove_unused_columns=False           # 찾았다 이새끼야
    )
)
_batched_data = batch_maker.get_test_dataloader(trainer_input)
batched_data = list(_batched_data)
single_batch = batched_data[-1]
model(**data_collator(single_batch))

ImageClassifierOutput(loss=tensor(4.7298, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[-0.0676,  0.1459, -0.1175, -0.1056, -0.0385,  0.0683, -0.1158, -0.1558,
          0.0525, -0.0200,  0.0376, -0.0063, -0.0155, -0.1117, -0.2427, -0.1596,
         -0.1851, -0.0097,  0.0415,  0.0059, -0.0049, -0.2989,  0.2192,  0.0929,
         -0.0221,  0.0580,  0.0327, -0.2149, -0.0519, -0.0122,  0.1131, -0.0620,
         -0.0930,  0.2360,  0.0717, -0.1277,  0.2066,  0.0503,  0.0279,  0.2441,
          0.0090,  0.2275,  0.0816,  0.0550, -0.0890, -0.0293,  0.0727, -0.2700,
          0.1172,  0.0787, -0.0140,  0.1297, -0.0702, -0.1394, -0.0369, -0.0392,
          0.1238, -0.0254,  0.1423, -0.1449, -0.1164, -0.1531, -0.0225,  0.0008,
          0.1119,  0.1147, -0.0927, -0.1213,  0.0792,  0.1642, -0.0733,  0.0432,
          0.0861,  0.1459,  0.1758, -0.0242,  0.1205, -0.0705, -0.0217, -0.0833,
         -0.0036,  0.0387,  0.0447,  0.1393,  0.0777,  0.0186,  0.0297,  0.0184,
          0.09

In [31]:
trainer = transformers.Trainer(
    model = model,
    data_collator= data_collator,
    args = transformers.TrainingArguments(            # triner에서도 똑같이 `args` 덩어리들 작성해주기
        output_dir="asdf",
        remove_unused_columns=False
    )
)
trainer.predict(trainer_input)

PredictionOutput(predictions=array([[ 0.08245477,  0.04544435, -0.02506349, ..., -0.09021501,
        -0.06471942,  0.01008125],
       [-0.08817462,  0.07300131,  0.0024789 , ..., -0.02228859,
        -0.14305958,  0.16596188],
       [ 0.07862606, -0.01995676,  0.05375681, ..., -0.13145116,
        -0.06238776,  0.01560452],
       ...,
       [-0.11510422,  0.02849287,  0.01068855, ..., -0.02876251,
        -0.05844339,  0.18115753],
       [-0.06757909,  0.14586404, -0.1175009 , ..., -0.01018211,
        -0.18151312,  0.11967795],
       [-0.19622399,  0.15283908, -0.07533113, ..., -0.01906088,
        -0.13519949,  0.03490356]], dtype=float32), label_ids=array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6]), metrics={'test_loss': 4.646465301513672, 'test_model_preparation_time': 0.0031, 'test_runtime': 0.1699, 'test_samples_per_second': 58.868, 'test_steps_per_second': 11.774})

풀이법 ㅈㄴ 많네 아

또 다른 풀이 방법:  트레이너가 가진 “사용하지 않는 column을 제거하는 기능”을 False 할 거면, `batch_maker`를 고려할 필요도 없이 아래와 같이 바로 `single_batch`를 얻을 수 있음.

"불필요한 컬럼 제거" 기능을 꺼두었으므로, 데이터의 모든 정보를 그대로 사용할 수 있다

따라서 Trainer 객체를 복잡하게 활용할 필요 없이 바로 데이터를 배치로 가져와 처리하면 된다

In [32]:
single_batch[-1]

{'label': 6,
 'pixel_values': tensor([[[ 0.2000,  0.2000,  0.1922,  ...,  0.4745,  0.4902,  0.4980],
          [ 0.2000,  0.2000,  0.1922,  ...,  0.4510,  0.4588,  0.4667],
          [ 0.2078,  0.2000,  0.1922,  ...,  0.4039,  0.4118,  0.4118],
          ...,
          [-0.0745, -0.0902, -0.1294,  ...,  0.2000,  0.2314,  0.2549],
          [-0.0980, -0.0980, -0.1137,  ...,  0.1608,  0.1686,  0.1765],
          [-0.1137, -0.1059, -0.0980,  ...,  0.1373,  0.1294,  0.1294]],
 
         [[-0.0510, -0.0510, -0.0588,  ...,  0.1765,  0.1922,  0.2000],
          [-0.0510, -0.0510, -0.0588,  ...,  0.1529,  0.1608,  0.1686],
          [-0.0431, -0.0510, -0.0588,  ...,  0.1059,  0.1137,  0.1137],
          ...,
          [-0.3490, -0.3647, -0.4039,  ..., -0.1765, -0.1451, -0.1294],
          [-0.3725, -0.3725, -0.3882,  ..., -0.2235, -0.2157, -0.2078],
          [-0.3882, -0.3804, -0.3725,  ..., -0.2471, -0.2549, -0.2549]],
 
         [[-0.1529, -0.1529, -0.1608,  ...,  0.1686,  0.1843,  0.1922],

In [33]:
trainer_input

Dataset({
    features: ['image', 'label'],
    num_rows: 10
})

## D. FOOD101 – DefaultDataCollator 구현

In [34]:
food = datasets.load_dataset("guebin/food101-tiny")
image_processor = transformers.AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
normalize = torchvision.transforms.Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop(size),
    torchvision.transforms.ToTensor(),
    normalize
])
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples
trainer_input = food['train'].with_transform(transforms)
trainer_input

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


Dataset({
    features: ['image', 'label'],
    num_rows: 10
})

In [35]:
labels = food["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label
model = transformers.AutoModelForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
# data_collator = transformers.DefaultDataCollator()
# data_collator

Q. `collate_fn(single_batch)` 을 직접 짜보세여ㅋㅋ

In [37]:
def collate_fn(single_batch):
    pass

In [38]:
trainer_input

Dataset({
    features: ['image', 'label'],
    num_rows: 10
})

먼저 `trainer_input`을 확인한다

이렇게 생겼군🧐

In [39]:
batch_maker = transformers.Trainer(
    model= model,
    data_collator= lambda x: x,
    args = transformers.TrainingArguments(
        output_dir="asdf",
        remove_unused_columns=False
    )
)
_batched_data = batch_maker.get_eval_dataloader(trainer_input)
batched_data = list(_batched_data)
single_batch = batched_data[-1]

위처럼 해도 되지만!

In [40]:
single_batch = [trainer_input[-2],trainer_input[-1]]
single_batch

[{'label': 6,
  'pixel_values': tensor([[[ 0.9529,  0.9059,  0.8745,  ...,  0.8588,  0.8745,  0.8588],
           [ 0.9686,  0.8980,  0.8275,  ...,  0.8667,  0.8824,  0.8510],
           [ 0.9765,  0.8824,  0.7569,  ...,  0.8588,  0.8353,  0.8510],
           ...,
           [ 0.7804,  0.7882,  0.7490,  ...,  0.8196,  0.8431,  0.7725],
           [ 0.7961,  0.7961,  0.7647,  ...,  0.7569,  0.7569,  0.7647],
           [ 0.8196,  0.8353,  0.8275,  ...,  0.7098,  0.7333,  0.7961]],
  
          [[ 0.8118,  0.7647,  0.6863,  ...,  0.6392,  0.6471,  0.6235],
           [ 0.8275,  0.7412,  0.6392,  ...,  0.6392,  0.6627,  0.6392],
           [ 0.8196,  0.6941,  0.5529,  ...,  0.6392,  0.6392,  0.6549],
           ...,
           [ 0.4510,  0.4353,  0.3804,  ...,  0.5137,  0.5451,  0.4980],
           [ 0.4510,  0.4431,  0.4118,  ...,  0.4510,  0.4745,  0.5059],
           [ 0.4745,  0.4980,  0.4902,  ...,  0.4118,  0.4588,  0.5294]],
  
          [[ 0.2706,  0.2078,  0.1373,  ...,  0.0667, 

이렇게 해도 똑같다!

아무튼 이렇게 `single_batch`를 만들어냈다

In [41]:
def collate_fn(single_batch):
    #single_batch = [Dict,Dict] 의 형식
    #Dict = {'label': 6, 'pixel_values': [3, 224, 224]-tensor
    collated_data = dict()
    collated_data['labels'] = torch.tensor([dct['label'] for dct in single_batch])
    collated_data['pixel_values'] = torch.stack([dct['pixel_values'] for dct in single_batch])
    return collated_data

`labels` = 모델이 분류해야 할 클래스 번호

`pixel_values` = 보통 이미지 데이터를 나타내며, 각 샘플은 다차원 텐서`(예: (채널, 높이, 너비))`로 구성

In [42]:
[dct['label'] for dct in single_batch]

[6, 6]

In [43]:
[dct['pixel_values'] for dct in single_batch]

[tensor([[[ 0.9529,  0.9059,  0.8745,  ...,  0.8588,  0.8745,  0.8588],
          [ 0.9686,  0.8980,  0.8275,  ...,  0.8667,  0.8824,  0.8510],
          [ 0.9765,  0.8824,  0.7569,  ...,  0.8588,  0.8353,  0.8510],
          ...,
          [ 0.7804,  0.7882,  0.7490,  ...,  0.8196,  0.8431,  0.7725],
          [ 0.7961,  0.7961,  0.7647,  ...,  0.7569,  0.7569,  0.7647],
          [ 0.8196,  0.8353,  0.8275,  ...,  0.7098,  0.7333,  0.7961]],
 
         [[ 0.8118,  0.7647,  0.6863,  ...,  0.6392,  0.6471,  0.6235],
          [ 0.8275,  0.7412,  0.6392,  ...,  0.6392,  0.6627,  0.6392],
          [ 0.8196,  0.6941,  0.5529,  ...,  0.6392,  0.6392,  0.6549],
          ...,
          [ 0.4510,  0.4353,  0.3804,  ...,  0.5137,  0.5451,  0.4980],
          [ 0.4510,  0.4431,  0.4118,  ...,  0.4510,  0.4745,  0.5059],
          [ 0.4745,  0.4980,  0.4902,  ...,  0.4118,  0.4588,  0.5294]],
 
         [[ 0.2706,  0.2078,  0.1373,  ...,  0.0667,  0.0980,  0.1294],
          [ 0.2863,  0.1686,

In [44]:
model.to("cpu")
model(**collate_fn(single_batch))

ImageClassifierOutput(loss=tensor(4.5832, grad_fn=<NllLossBackward0>), logits=tensor([[ 0.0036,  0.1137,  0.1630, -0.0265, -0.1198,  0.0047, -0.0716,  0.0433,
         -0.0668,  0.0821, -0.0274,  0.0466, -0.1202, -0.1752,  0.1690, -0.0296,
         -0.0654,  0.0335,  0.0271,  0.1060, -0.0170, -0.0306,  0.1520, -0.0370,
          0.1018,  0.0066, -0.0116, -0.0980,  0.0301, -0.1715, -0.0429, -0.1252,
         -0.2231, -0.0736,  0.0674,  0.0184, -0.0143,  0.0758,  0.0029, -0.1160,
          0.0182,  0.1285, -0.0670,  0.0342, -0.1104, -0.0173,  0.0310,  0.0343,
         -0.0022,  0.0218, -0.0311,  0.0192,  0.0177,  0.1248,  0.1798, -0.0771,
          0.0871,  0.0903,  0.2927,  0.1053,  0.1109, -0.2903,  0.0980, -0.2225,
         -0.1984, -0.0604,  0.1315, -0.1846,  0.0015, -0.0685, -0.0130,  0.0190,
          0.1221, -0.1196, -0.1463, -0.0587, -0.0869,  0.0506,  0.0055,  0.0147,
         -0.1946, -0.1194,  0.1065, -0.0269,  0.0632,  0.0268, -0.0333,  0.0524,
         -0.0687, -0.0809, -0.0

In [45]:
trainer = transformers.Trainer(
    model=model,
    data_collator=collate_fn,
    args=transformers.TrainingArguments(
        output_dir="asdf",
        remove_unused_columns=False
    )
)
trainer.predict(trainer_input)

PredictionOutput(predictions=array([[ 0.12243795,  0.06259688,  0.04917661, ...,  0.04959228,
        -0.03346272,  0.17124057],
       [ 0.13723229,  0.116909  ,  0.06919269, ...,  0.08047475,
         0.04463541,  0.13185509],
       [ 0.12723413,  0.12039883,  0.1084886 , ...,  0.10231652,
         0.03938451,  0.09179737],
       ...,
       [-0.05683486,  0.23519842,  0.12504767, ...,  0.11399097,
        -0.05379739,  0.04120644],
       [ 0.02872899,  0.09113608,  0.20390931, ...,  0.01311333,
         0.02106166,  0.01048891],
       [ 0.17461318,  0.11315147,  0.12171987, ...,  0.00803177,
         0.00615833,  0.04904607]], dtype=float32), label_ids=array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6]), metrics={'test_loss': 4.569972991943359, 'test_model_preparation_time': 0.0083, 'test_runtime': 0.2379, 'test_samples_per_second': 42.038, 'test_steps_per_second': 8.408})

## E. IMDB – DataCollatorWithPadding 구현

In [46]:
imdb = datasets.load_dataset("guebin/imdb-tiny")
tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)
tokenized_imdb = imdb.map(preprocess_function,batched=True)
trainer_input = tokenized_imdb['train']

In [47]:
model = transformers.AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Q. 이번에도 네가 `data_collator` 만들어ㅋㅋ

In [48]:
# data_collator = transformers.DataCollatorWithPadding(tokenizer=tokenizer)
# data_collator

# 랑 똑같은 `collate_fn`을 만들어보세여

In [49]:
def collate_fn(single_batch):
    pass

In [50]:
trainer_input

# 더 많은 데이터가 생겼군🧐

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 10
})

In [51]:
batch_maker = transformers.Trainer(
    model= model,
    data_collator= lambda x: x,
    args = transformers.TrainingArguments(
        output_dir="asdf",
        remove_unused_columns=False
    )
)
_batched_data = batch_maker.get_eval_dataloader(trainer_input)
batched_data = list(_batched_data)
single_batch = batched_data[-1]

In [52]:
single_batch[0].keys()

dict_keys(['text', 'label', 'input_ids', 'attention_mask'])

In [53]:
# single_batch = [Dict, Dict]
# Dict = {
#     'label': int
#     'input_ids': 1d-list
#     'attention_mask': 1d-list
# }
def collate_fn(single_batch):
    collated_data = dict()             # 이렇게 시작하기 때문에 return값이 dict이게 됨
    collated_data['labels'] = torch.tensor([dct['label'] for dct in single_batch])
    collated_data['input_ids'] = torch.nn.utils.rnn.pad_sequence([torch.tensor(dct['input_ids']) for dct in single_batch]).t()
    collated_data['attention_mask'] = torch.nn.utils.rnn.pad_sequence([torch.tensor(dct['attention_mask']) for dct in single_batch]).t()
    return collated_data

In [54]:
[len(dct['input_ids']) for dct in single_batch]

# 길이가 다르기 때문에 `padding` 기능 必
# 왠지 시험 볼 때도 `padding` 기능 있어야 할 것 같은 기분,,,

[512, 297]

In [55]:
torch.nn.utils.rnn.pad_sequence([torch.tensor(dct['input_ids']) for dct in single_batch]).t()

# `padding`이 실행되었다
# `input_ids`와 `attention_mask`의 함수(?) 는 동일하

tensor([[  101,  2040,  2024,  ..., 22132,  7847,   102],
        [  101,  2023,  2003,  ...,     0,     0,     0]])

In [56]:
model.to("cpu")
model(**collate_fn(single_batch))

SequenceClassifierOutput(loss=tensor(0.6734, grad_fn=<NllLossBackward0>), logits=tensor([[ 0.0025, -0.0352],
        [ 0.0139, -0.0282]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [57]:
trainer = transformers.Trainer(
    model=model,
    data_collator=collate_fn,
)
trainer.predict(trainer_input)

PredictionOutput(predictions=array([[ 0.08966784, -0.06166392],
       [ 0.01500151, -0.06875469],
       [ 0.03916246, -0.00611847],
       [ 0.04056273, -0.05440576],
       [-0.00634491, -0.03747641],
       [ 0.03792845, -0.04466521],
       [ 0.03690896, -0.08592637],
       [ 0.00534204, -0.05428017],
       [ 0.00248964, -0.03522495],
       [ 0.01390313, -0.02818928]], dtype=float32), label_ids=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), metrics={'test_loss': 0.6564630270004272, 'test_model_preparation_time': 0.0015, 'test_runtime': 0.1983, 'test_samples_per_second': 50.421, 'test_steps_per_second': 10.084})

In [58]:
trainer_input

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 10
})

In [59]:
trainer = transformers.Trainer(
    model=model,
    data_collator=collate_fn,
    args=transformers.TrainingArguments(
        output_dir="asdf",
        remove_unused_columns=False
    )
)
trainer.predict(trainer_input)

PredictionOutput(predictions=array([[ 0.08966784, -0.06166392],
       [ 0.01500151, -0.06875469],
       [ 0.03916246, -0.00611847],
       [ 0.04056273, -0.05440576],
       [-0.00634491, -0.03747641],
       [ 0.03792845, -0.04466521],
       [ 0.03690896, -0.08592637],
       [ 0.00534204, -0.05428017],
       [ 0.00248964, -0.03522495],
       [ 0.01390313, -0.02818928]], dtype=float32), label_ids=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), metrics={'test_loss': 0.6564630270004272, 'test_model_preparation_time': 0.0014, 'test_runtime': 0.1305, 'test_samples_per_second': 76.618, 'test_steps_per_second': 15.324})

In [60]:
trainer_input

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 10
})

`text`가 들어있지 않은데도 `trainer.predict(trainer_input)`이 가능한 이유:

`collate_fn` 을 정의할 때, 사용된 요소는 `input_ids`, `attention_mask`, `labels` 뿐이기 때문

```python
trainer = transformers.Trainer(
    model=model,
    data_collator=collate_fn
)
```

라고 작성해도 실행된다

In [61]:
trainer = transformers.Trainer(
    model=model,
    data_collator=collate_fn
)

trainer.predict(trainer_input)

# 짠!

PredictionOutput(predictions=array([[ 0.08966784, -0.06166392],
       [ 0.01500151, -0.06875469],
       [ 0.03916246, -0.00611847],
       [ 0.04056273, -0.05440576],
       [-0.00634491, -0.03747641],
       [ 0.03792845, -0.04466521],
       [ 0.03690896, -0.08592637],
       [ 0.00534204, -0.05428017],
       [ 0.00248964, -0.03522495],
       [ 0.01390313, -0.02818928]], dtype=float32), label_ids=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), metrics={'test_loss': 0.6564630270004272, 'test_model_preparation_time': 0.0015, 'test_runtime': 0.1258, 'test_samples_per_second': 79.489, 'test_steps_per_second': 15.898})

# 4. 연습 – sms_spam


In [62]:
model = transformers.AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2
)
tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
spam = datasets.load_dataset('guebin/spam-tiny')
spam

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


README.md:   0%|          | 0.00/362 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.29k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sms', 'label'],
        num_rows: 10
    })
})

In [63]:
spam

DatasetDict({
    train: Dataset({
        features: ['sms', 'label'],
        num_rows: 10
    })
})

## A. 방법1: 고정패딩, `collate_fn`

In [64]:
def m_trans(example_batch):
    # example_batch = {'sms':[xxx,xxxx,...], 'label':[yyy,yyyy]
    # example_batch = spam['train'][:8]
    # out = tokenizer(example_batch['sms'],padding=True,truncation=True)
    return out

In [71]:
spam2 = spam.map(m_trans,batched=True,batch_size=8)
spam2

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sms', 'label', 'input_ids', 'attention_mask'],
        num_rows: 10
    })
})

In [72]:
spam.map(m_trans)['train'][0]
# `padding`이 안 됨!!
# 그래서 batch_size를 지정해줘야대

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

{'sms': 'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...\n',
 'label': 0,
 'input_ids': [101,
  2175,
  2127,
  18414,
  17583,
  2391,
  1010,
  4689,
  1012,
  1012,
  2800,
  2069,
  1999,
  11829,
  2483,
  1050,
  2307,
  2088,
  2474,
  1041,
  28305,
  1012,
  1012,
  1012,
  25022,
  2638,
  2045,
  2288,
  26297,
  28194,
  1012,
  1012,
  1012,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [73]:
def m_trans(example_batch):
    # example_batch = {'sms':[xxx,xxxx,...], 'label':[yyy,yyyy]
    # example_batch = spam['train'][:8]
    out = tokenizer(example_batch['sms'],padding=True,truncation=True)
    return out

In [74]:
[len(l) for l in spam2['train']['input_ids']]
# `padding`이 잘 되었군,,,

[56, 56, 56, 56, 56, 56, 56, 56, 46, 46]

In [75]:
spam2.set_format("pt")
#spam2['train']['input_ids'] -- list of tensor with length 10

In [76]:
spam2['train'][:8]['label'] # 1d-tensor

tensor([0, 0, 1, 0, 0, 1, 0, 0])

In [83]:
spam2['train'][7:]['input_ids'] # 1d-tensor

# 왜냐하면 batch_size 이하이기 때문

[tensor([  101,  2004,  2566,  2115,  5227,  1005, 11463,  2571, 11463,  2571,
          1006,  2030,  2226,  8117, 28987, 11231,  3070, 18447,  2063, 27617,
          5575,  2226, 29525, 15464,  1007,  1005,  2038,  2042,  2275,  2004,
          2115, 20587,  8525,  2638,  2005,  2035, 20587,  2015,  1012,  2811,
          1008,  1023,  2000,  6100,  2115,  2814, 20587,  8525,  2638,   102,
             0,     0,     0,     0,     0,     0]),
 tensor([  101,  3453,   999,   999,  2004,  1037, 11126,  2897,  8013,  2017,
          2031,  2042,  3479,  2000,  4374,  2050,  1069, 21057,  2692,  3396,
         10377,   999,  2000,  4366,  2655,  5641,  2692,  2575, 16576, 24096,
         21472,  2487,  1012,  4366,  3642,  1047,  2140, 22022,  2487,  1012,
          9398,  2260,  2847,  2069,  1012,   102]),
 tensor([  101,  2018,  2115,  4684,  2340,  2706,  2030,  2062,  1029,  1057,
          1054,  4709,  2000, 10651,  2000,  1996,  6745,  6120,  4684,  2015,
          2007,  4950,  2

In [84]:
spam2['train'][:8]['input_ids'] # 2d-tensor

# batch_size의 크기를 넘어갔으므로 2d-tensor(당연한 거 아닌가)

tensor([[  101,  2175,  2127, 18414, 17583,  2391,  1010,  4689,  1012,  1012,
          2800,  2069,  1999, 11829,  2483,  1050,  2307,  2088,  2474,  1041,
         28305,  1012,  1012,  1012, 25022,  2638,  2045,  2288, 26297, 28194,
          1012,  1012,  1012,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0],
        [  101,  7929,  2474,  2099,  1012,  1012,  1012, 16644, 15536,  2546,
          1057,  2006,  2072,  1012,  1012,  1012,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0],
        [  101,  2489,  4443,  1999,  1016,  1037,  1059,  2243,  2135,  4012,
          2361,  2000,  266

In [85]:
trainer_input = spam2['train'].remove_columns(['sms']).rename_columns({'label':'labels'})
trainer_input

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 10
})

In [91]:
batch_maker = transformers.Trainer(
    model= model,
    data_collator=lambda x:x
)
_batched_data = batch_maker.get_eval_dataloader(trainer_input)
batched_data = list(_batched_data)
single_batch = batched_data[-1]
single_batch

[{'labels': tensor(1, device='cuda:0'),
  'input_ids': tensor([  101,  3453,   999,   999,  2004,  1037, 11126,  2897,  8013,  2017,
           2031,  2042,  3479,  2000,  4374,  2050,  1069, 21057,  2692,  3396,
          10377,   999,  2000,  4366,  2655,  5641,  2692,  2575, 16576, 24096,
          21472,  2487,  1012,  4366,  3642,  1047,  2140, 22022,  2487,  1012,
           9398,  2260,  2847,  2069,  1012,   102], device='cuda:0'),
  'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         device='cuda:0')},
 {'labels': tensor(1, device='cuda:0'),
  'input_ids': tensor([  101,  2018,  2115,  4684,  2340,  2706,  2030,  2062,  1029,  1057,
           1054,  4709,  2000, 10651,  2000,  1996,  6745,  6120,  4684,  2015,
           2007,  4950,  2005,  2489,   999,  2655,  1996,  4684, 10651,  2522,
           2489,  2006,  5511,  8889, 24594, 20842,  2692,

이런 형태로는 `model`에 넣지 못 함

`torch` 및 `stack` 실시 = `collate_fn` 설계

In [94]:
# torch.stack([single_batch[0]['labels'],single_batch[1]['labels']])

In [92]:
def collate_fn(single_batch):
    out = dict()
    out['labels'] = torch.stack([dct['labels'] for dct in single_batch])
    out['input_ids'] = torch.stack([dct['input_ids'] for dct in single_batch])
    out['attention_mask'] = torch.stack([dct['attention_mask'] for dct in single_batch])
    return out

In [93]:
model(**collate_fn(single_batch))

SequenceClassifierOutput(loss=tensor(0.6834, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[-0.0209, -0.0154],
        [-0.0153,  0.0186]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

## B. 방법2: 고정패딩, DefaultDataCollator

In [132]:
def m_trans(example_batch):
    # example_batch = {'sms':[xxx,xxxx,...], 'label':[yyy,yyyy]
    # example_batch = spam['train'][:8]
    out = tokenizer(example_batch['sms'],padding=True,truncation=True)
    return out
spam2 = spam.map(m_trans,batched=True,batch_size=8)
spam2.set_format("pt")
trainer_input = spam2['train'].remove_columns(['sms']).rename_columns({'label':'labels'})

In [133]:
batch_maker = transformers.Trainer(
    model= model,
    data_collator=lambda x:x
)
_batched_data = batch_maker.get_eval_dataloader(trainer_input)
batched_data = list(_batched_data)
single_batch = batched_data[-1]
single_batch

[{'labels': tensor(1, device='cuda:0'),
  'input_ids': tensor([  101,  3453,   999,   999,  2004,  1037, 11126,  2897,  8013,  2017,
           2031,  2042,  3479,  2000,  4374,  2050,  1069, 21057,  2692,  3396,
          10377,   999,  2000,  4366,  2655,  5641,  2692,  2575, 16576, 24096,
          21472,  2487,  1012,  4366,  3642,  1047,  2140, 22022,  2487,  1012,
           9398,  2260,  2847,  2069,  1012,   102], device='cuda:0'),
  'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         device='cuda:0')},
 {'labels': tensor(1, device='cuda:0'),
  'input_ids': tensor([  101,  2018,  2115,  4684,  2340,  2706,  2030,  2062,  1029,  1057,
           1054,  4709,  2000, 10651,  2000,  1996,  6745,  6120,  4684,  2015,
           2007,  4950,  2005,  2489,   999,  2655,  1996,  4684, 10651,  2522,
           2489,  2006,  5511,  8889, 24594, 20842,  2692,

In [134]:
# def collate_fn(single_batch):
#     out = dict()
#     out['labels'] = torch.stack([dct['labels'] for dct in single_batch])
#     out['input_ids'] = torch.stack([dct['input_ids'] for dct in single_batch])
#     out['attention_mask'] = torch.stack([dct['attention_mask'] for dct in single_batch])
#     return out
data_collator = transformers.DefaultDataCollator()

In [135]:
# data_collator 함수 활용
model(**data_collator(single_batch))

SequenceClassifierOutput(loss=tensor(0.2443, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[-0.5512,  0.6907],
        [-0.6280,  0.7012]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [136]:
# collate_fn(single_batch) 함수 활용
model.to("cpu")
model(**collate_fn(single_batch))

# 결과가 똑같습니듀

KeyError: 'sms'

In [139]:
trainer = transformers.Trainer(
    model=model,
    data_collator=data_collator,
    train_dataset=trainer_input,
    args = transformers.TrainingArguments(
        output_dir="asdf",
        remove_unused_columns=False
    )
)
# trainer.train()

In [140]:
trainer = transformers.Trainer(
    model= model,
    data_collator=data_collator
)
trainer.predict(trainer_input)

PredictionOutput(predictions=array([[ 0.9782646 , -1.0919646 ],
       [ 1.0289698 , -1.0795335 ],
       [-0.6568082 ,  0.72002107],
       [ 1.0936704 , -1.0427283 ],
       [ 0.7971047 , -0.85264856],
       [-0.6762122 ,  0.6755601 ],
       [ 0.5328952 , -0.49847734],
       [ 0.6644742 , -0.739807  ],
       [-0.5511835 ,  0.6907274 ],
       [-0.628031  ,  0.70117867]], dtype=float32), label_ids=array([0, 0, 1, 0, 0, 1, 0, 0, 1, 1]), metrics={'test_loss': 0.1988973319530487, 'test_model_preparation_time': 0.0029, 'test_runtime': 0.0397, 'test_samples_per_second': 251.892, 'test_steps_per_second': 50.378})

## C. 방법3: 동적패딩,`DataCollatorWithPadding`

In [103]:
spam

DatasetDict({
    train: Dataset({
        features: ['sms', 'label'],
        num_rows: 10
    })
})

In [144]:
def w_trans(examples):
    # examples = spam['train'][:8] = {'sms': [xxx,xxxx,...], 'label':[yyy,yyyy,...]
    out = tokenizer(examples['sms'],truncation=True)
    out['labels'] = torch.tensor(examples['label'])
    return out

In [145]:
trainer_input = spam.with_transform(w_trans)['train']
trainer_input

Dataset({
    features: ['sms', 'label'],
    num_rows: 10
})

In [154]:
# trainer_input[0]
# `padding` 없이 잘 나옵니다(아직 `padding` 안 함)

In [157]:
batch_maker = transformers.Trainer(
    model = model,
    data_collator = lambda x: x,
    args = transformers.TrainingArguments(
        output_dir="asdf",
        remove_unused_columns=False
    )
)
single_batch = next(iter(batch_maker.get_eval_dataloader(trainer_input))) # (with)transform 기능을 사용하기 때문에
                                                                          # 이 코드가 데이터값이 사라지는 걸 막아줌
# single_batch
# 아직도 `padding` 기능 안 넣은 거 맞습니다

In [158]:
data_collator = transformers.DataCollatorWithPadding(tokenizer) # 지금 `padding` 기능 추가함
model.to("cpu")
model(**data_collator(single_batch))

SequenceClassifierOutput(loss=tensor(0.1876, grad_fn=<NllLossBackward0>), logits=tensor([[ 0.9783, -1.0920],
        [ 1.0290, -1.0795],
        [-0.6568,  0.7200],
        [ 1.0937, -1.0427],
        [ 0.7971, -0.8526],
        [-0.6762,  0.6756],
        [ 0.5329, -0.4985],
        [ 0.6645, -0.7398]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [159]:
trainer = transformers.Trainer(
    model = model,
    data_collator = data_collator,
    args = transformers.TrainingArguments(
        output_dir="asdf",
        remove_unused_columns=False
    )
)
trainer.predict(trainer_input)

PredictionOutput(predictions=array([[ 0.9782646 , -1.0919646 ],
       [ 1.0289698 , -1.0795335 ],
       [-0.6568082 ,  0.72002107],
       [ 1.0936704 , -1.0427283 ],
       [ 0.7971047 , -0.85264856],
       [-0.6762122 ,  0.6755601 ],
       [ 0.5328952 , -0.49847734],
       [ 0.6644742 , -0.739807  ],
       [-0.5511835 ,  0.6907274 ],
       [-0.628031  ,  0.70117867]], dtype=float32), label_ids=array([0, 0, 1, 0, 0, 1, 0, 0, 1, 1]), metrics={'test_loss': 0.1988973319530487, 'test_model_preparation_time': 0.0014, 'test_runtime': 0.0377, 'test_samples_per_second': 265.44, 'test_steps_per_second': 53.088})

In [160]:
trainer = transformers.Trainer(
    model=model,
    data_collator=data_collator,
    train_dataset=trainer_input,
    args = transformers.TrainingArguments(
        output_dir="asdf",
        remove_unused_columns=False
    )
)
trainer.train()

Step,Training Loss


TrainOutput(global_step=6, training_loss=0.10676061113675435, metrics={'train_runtime': 17.5344, 'train_samples_per_second': 1.711, 'train_steps_per_second': 0.342, 'total_flos': 421204931664.0, 'train_loss': 0.10676061113675435, 'epoch': 3.0})

## D. 방법4: 동적패딩, 전처리X (★)

In [110]:
trainer_input = spam['train']
trainer_input

Dataset({
    features: ['sms', 'label'],
    num_rows: 10
})

In [111]:
single_batch = [trainer_input[-2],trainer_input[-1]]
single_batch

[{'sms': 'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.\n',
  'label': 1},
 {'sms': 'Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030\n',
  'label': 1}]

In [112]:
def collate_fn(single_batch):
    out = tokenizer(
        [dct['sms'] for dct in single_batch],
        padding=True,
        truncation=True,
        return_tensors="pt",
    )
    out['labels'] = torch.tensor([dct['label'] for dct in single_batch])
    return out

In [113]:
model.to("cpu")
model(**collate_fn(single_batch))

SequenceClassifierOutput(loss=tensor(0.5701, grad_fn=<NllLossBackward0>), logits=tensor([[-0.1743,  0.0663],
        [-0.1403,  0.1460]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [114]:
trainer = transformers.Trainer(
    model=model,
    data_collator=collate_fn,
    args = transformers.TrainingArguments(
        output_dir="asdf",
        remove_unused_columns=False
    )
)
trainer.predict(trainer_input)

PredictionOutput(predictions=array([[ 0.32142335, -0.37574598],
       [ 0.40545943, -0.43210134],
       [-0.1081787 ,  0.12635195],
       [ 0.47563726, -0.40755242],
       [ 0.30406964, -0.34332997],
       [-0.06769809,  0.04940781],
       [ 0.05315341, -0.1912716 ],
       [ 0.08524358, -0.23379186],
       [-0.10524124,  0.13367648],
       [-0.08915347,  0.13477877]], dtype=float32), label_ids=array([0, 0, 1, 0, 0, 1, 0, 0, 1, 1]), metrics={'test_loss': 0.5042697191238403, 'test_model_preparation_time': 0.0026, 'test_runtime': 0.0439, 'test_samples_per_second': 227.563, 'test_steps_per_second': 45.513})

In [115]:
trainer = transformers.Trainer(
    model=model,
    data_collator=collate_fn,
    train_dataset=trainer_input,
    args = transformers.TrainingArguments(
        output_dir="asdf",
        remove_unused_columns=False
    )
)
trainer.train()

Step,Training Loss


TrainOutput(global_step=6, training_loss=0.30615750948588055, metrics={'train_runtime': 12.1566, 'train_samples_per_second': 2.468, 'train_steps_per_second': 0.494, 'total_flos': 421204931664.0, 'train_loss': 0.30615750948588055, 'epoch': 3.0})